In [1]:
# MHAR walk

In [2]:
#| default_exp walk

In [3]:
#| export
import torch

from typing import Union
from tqdm import tqdm

from mhar.polytope import Polytope, NFDPolytope
from mhar.utils import print_progress

In [4]:
#| export

def sample_inner_points(X0, z):
    n, k = X0.shape
    if z > k:
        # Pad by repeating vectors from X to fill z columns
        num_repeats = z // k
        remainder = z % k
        I = torch.cat([X0] * num_repeats + [X0[:, :remainder]], dim=1)
    elif z < k:
        # Take the first z columns from X
        I = X0[:, :z]
    else:
        # z == k, I equals X
        I = X0.clone()
    return I

In [5]:
#| export

def create_h(n,z,generator,dtype,device):
    """
    Creates a Tensor (z x n x 1) where each entry ~ N(0,1). Automatically detects the
    precision 64 bits or 32 bits.
    -------------
    :param n:   int
                Dimension of the Space where the Polytope Lives
    :param z:   int
                Padding Parameter
    :param generator:
    :param device:  String, default = cpu
                    Hardware used to make the computations and allocate the result.
                    If equal to cpu then the CPUs are used for computing the inverse.
                    If equal to cuda then the a GPU is used for computing the inverse.
    -------------
    :return:    Torch Tensor
                Tensor (z x n x 1) where each entry ~ N(0,1)Contains a tensor

    """
    if '64' in str(dtype):
        if 'cuda' in device:
            h = torch.cuda.DoubleTensor(n, z).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.DoubleTensor(n, z).normal_(generator=generator)
    elif '32' in str(dtype):
        if 'cuda' in device:
            h = torch.cuda.FloatTensor(n, z).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.FloatTensor(n, z).normal_(generator=generator)
    elif '16' in str(dtype):
        if 'cuda' in device:
            h = torch.cuda.HalfTensor(n, z).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.HalfTensor(n, z).normal_(generator=generator)

    return h

In [6]:
#| export

def draw_uniform(z, generator,dtype,device='cpu'):
    """
    Creates a tensor (z x 1) where each entry ~ U(0,1). Automatically detects the
    precision 64 bits or 32 bits.
    -------------
    :param n:   int
                Dimension of the Space where the Polytope Lives
    :param z:   int
                Padding Parameter
    :param generator:
    :param device:  String, default = cpu
                    Hardware used to make the computations and allocate the result.
                    If equal to cpu then the CPUs are used for computing the inverse.
                    If equal to cuda then the a GPU is used for computing the inverse.
    -------------
    :return:    Torch Tensor
                Tensor (z x 1) where each entry ~ U(0,1)

    """

    if '64' in str(dtype):
        if 'cuda' in device:
            u = torch.cuda.DoubleTensor(1, z).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.DoubleTensor(1,z).uniform_(generator=generator)
    elif '32' in str(dtype):
        if 'cuda' in device:
            u = torch.cuda.FloatTensor(1, z).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.FloatTensor(1, z).uniform_(generator=generator)
    elif '16' in str(dtype):
        if 'cuda' in device:
            u = torch.cuda.HalfTensor(1, z).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.HalfTensor(1, z).uniform_(generator=generator)

    return u
    

In [7]:
#| export
import sys

def update_progress_bars(burned, thinning, t, T, new_t: bool = False):
    # Generate progress bar string for burned samples
    if thinning !=0:
        progress_burned = print_progress(iteration=burned, prefix='% of burned samples', total=thinning, length=30)

    # Clear the current line
    sys.stdout.write('\033[K')

    if new_t:
        # Generate progress bar string for iid samples
        progress_iid = print_progress(iteration=t, prefix='% of iid samples', total=T, length=30)
        # Print both progress bars
        if thinning !=0:
            print(progress_burned, end='\n')
        print(progress_iid, end='\n')
        # Move cursor up two lines for the next update
        #sys.stdout.write("\033[F\033[F")
    else:
        # Print only the progress bar for burned samples
        if thinning !=0:
            print(progress_burned, end='\r')

    # If new_t is False, the cursor will remain on the same line ready for the next update


In [91]:

#| export

def walk(
        polytope:Union[Polytope, NFDPolytope], # Polytope Object
        X0:torch.Tensor,    # Initial Interior point(s) of dim=(n,k).     
                            #- If z > k (number of columns in X0), pad  by repeating vectors from X to fill z columns.
                            #- If z < k, take the first z columns from X0.
                            #- If z == k, equal to X0.
        z:int=1, # The number of simultaneous to be executed (padding parameter).
        T:int=1, # id-iterations, total_iid_points = T*z. Each iid iteration will burn the samples established by the thinning factor.
        warm:int=None, # Number of iid-iterations needed to warm. The walk will execute warm steps before saving the points.
        thinning:int=None, # Thinning Factor. Default O(n^3)
        device:str = None, # Deveice to use, cpu or cuda
        seed:int=None, # Seed for Pseudo-Random Number Generation
        verbosity:int=1, # Verbosity of the execution
        ) -> torch.Tensor:
        
        
    ## Check validity 
    # Device
    device = device if device is not None else polytope.device
    assert(device in ['cpu', 'cuda']), print('The device is not correctly specified: ', device,
                                        '\n Please choose cpu or cuda')
    # X0 dimension
    assert(X0.shape[0] == polytope.n)
    
    ## Set min and max values
    min_ = torch.finfo(polytope.dtype).min + 2
    max_ = torch.finfo(polytope.dtype).max - 2
    eps =  torch.finfo(polytope.dtype).eps
    if verbosity > 1:
        print(f'Minimum number allowed {min_}')
        print(f'Maximum number allowed {max_}')
    
    
    ## Set seed
    random_gen = torch.Generator(device=device)
    if seed:
        random_gen.manual_seed(seed)
    else:
        random_gen.seed()
    
    
    ## Check Dimensions
    n = polytope.n
    mI = polytope.mI
    if isinstance(polytope, NFDPolytope) :
        mE = polytope.mE
    else:
        mE=None            
    if verbosity >=1:
        print('n: ', n, '  mI:', mI, '  mE:', mE, '  z:', z)
        
    
    ## Compute/set thinning factor
    if thinning is not None:
        pass
    else:
        thinning = int(n * n * n)
        if verbosity >= 1:
            print('Automatic Thinning factor: ', thinning)
            
    if warm is not None:
        pass
    else:
        warm = thinning
        if verbosity >= 1:
            print('Warming iterations: ', thinning)
     
            
    ## Prepare and send Matrices
    init_x0 = sample_inner_points(X0,z).to(device)  
    polytope.send_to_device(device)
    
    ## Iteration Loop
    t = 1
    burned = 0
    dtype = polytope.dtype
    x = init_x0
    X = torch.empty((T,n, z))
    while t <= T:
        h = create_h(n, z, generator=random_gen, dtype=dtype,device=device)
        if isinstance(polytope, NFDPolytope) :
            d = polytope.project_h(h)
        else:
            d = h
            
        ## Compute B - AX / AD
        numerator = polytope.b_in - torch.matmul(polytope.A_in, x)
        denominator = torch.matmul(polytope.A_in, d)
        
        # Overwrite numerator to keep memory free
        numerator = numerator / denominator
        # From the positive denominators you want the smallest one
        lambda_pos = torch.min((denominator < 0.0).to(dtype) * max_ + # For negative denominator use max val
                               (denominator > 0.0).to(dtype) * numerator, 0).values.to(dtype)[None,:]
        # From the negative denominators you want the biggest
        lambda_neg = torch.max((denominator > 0.0).to(dtype) * min_ +
                               (denominator < 0.0).to(dtype) * numerator, 0).values.to(dtype)[None,:]
        
    
        # Uniform draw
        u = draw_uniform(z, generator=random_gen, dtype=dtype,device=polytope.device)

        
        theta = (1.0 - u) * lambda_pos + u * lambda_neg

        
        # New X
        x = x + d * theta
        if verbosity >=1:
            update_progress_bars(burned, thinning, t, T)        
        
        if (warm > 0) & (burned >= thinning):
            warm = warm - 1
            burned = 0
        
        # Manage the burning rate and save points
        if burned >= thinning:
            X[t-1,:, :] = x.to('cpu')        
            
            
            if verbosity >=1:
                update_progress_bars(burned, thinning, t, T, new_t=True)
            t = t + 1
            burned = 0

        burned = burned + 1

    return X

    

In [9]:
from mhar.polytope_examples import Hypercube
from mhar.inner_point import ChebyshevCenter

import numpy as np
import torch

hypercube = Hypercube(3,
                      dtype=torch.float16,
                      device='cuda'
                      )
x0 = ChebyshevCenter(polytope=hypercube, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device='cuda',
                    solver_precision=np.float64)

X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=100, 
        warm=0,
        thinning=None, 
        device= None, 
        seed=None,
        verbosity=1
)

/home/uumami/sonder.art/mhar/mhar/polytope.py:97: UserWarning:
  The dtype torch.float16 is typically used with GPU architectures. If you are using CPU, consider
  using 32 or 64-bit dtypes. Certain operations may be casted to 32 or 64 bits to enhance numerical
  stability.

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.
n:  3   mI: 6   mE: None   z: 100
Automatic Thinning factor:  27
/tmp/ipykernel_115130/1586572147.py:34: UserWarning:
  The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such
  as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at
  ../torch/csrc/tensor/python_tensor.cpp:83.)

% of burned samples |██████████████████████████████| 100.0%
% of iid samples |------------------------------| 1.0%
% of burne

In [14]:
(X.abs()>1).sum()

tensor(0)

In [95]:
from mhar.polytope_examples import Simplex
from mhar.inner_point import ChebyshevCenter

import numpy as np
import torch

simplex = Simplex(
    n=10000,
    dtype=torch.float32,
    copy=False,
    device='cuda',
    requires_grad=False
)
simplex.compute_projection_matrix(device='cuda')

x0 = ChebyshevCenter(polytope=simplex, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-10,
                    device='cuda',
                    solver_precision=np.float64)

X = walk(polytope=simplex,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=1000, 
        device= None, 
        seed=None,
        verbosity=2
)

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object

Max non zero error for term (A A')^(-1)A at precision torch.float32:  tensor(5.9605e-08, device='cuda:0')

Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.
Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
n:  10000   mI: 10000   mE: 1   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


In [96]:
X.sum(1)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000]])